In [8]:
from tic_tac_toe import UltimateBoard, ClassicBoard
import pandas as pd
import numpy as np

import seaborn as sns
from matplotlib import pyplot as plt
plt.style.use('ggplot')
%matplotlib inline

colors=['#547fc4']

import keras

from multiprocessing import Pool
from time import time

In [9]:
game = UltimateBoard()

In [10]:
def sim_game(i):
    x_turn = i % 2 == 0
    board = UltimateBoard(x_turn)
    turns = 0
    
    np.random.seed()
    
    while not board.game_over:

            lab = 'X' if x_turn else 'O'
            brd = np.random.choice(list(board.active_boards), size=1)[0]
            pos = np.random.choice(list(board.boards[brd].remaining_squares), size=1)[0]

            board.play(brd, pos, lab)

            x_turn = not x_turn
            turns += 1
    
    df = pd.DataFrame(board.get_history())
    df['game'] = i
    return (df, board)

In [11]:
n_games = 10000

In [12]:
tic = time()
p = Pool(4)
res = p.map(sim_game, range(n_games))
print(time() - tic)

73.54706478118896


In [ ]:
boards = [r[1] for r in res]
dfs = [r[0] for r in res]

In [ ]:
full_res = pd.concat(dfs, sort=True)

In [ ]:
winner_map = {'Cat':0, 'X':1, 'O':2}
full_res['winner'] = full_res[90].map(winner_map)
full_res['x_wins'] = (full_res[90] == 'X').astype(int)

In [ ]:
from sklearn.model_selection import train_test_split
train_games, val_test_games = train_test_split(list(range(n_games)), test_size=.25)
val_games, test_games = train_test_split(val_test_games, test_size=.5)

In [ ]:
train = full_res[full_res['game'].isin(train_games)].copy().reset_index(drop=True)
val = full_res[full_res['game'].isin(val_games)].copy().reset_index(drop=True)
test = full_res[full_res['game'].isin(test_games)].copy().reset_index(drop=True)

In [ ]:
from keras.models import Sequential
from keras.layers import Dense, Activation, Conv2D, ReLU, Dropout
from keras.layers import Flatten, MaxPooling2D

In [ ]:
train_X = train.values[:,:81]
train_X = train_X.reshape((train.shape[0], 9, 9, 1))

train_y = np.array(train['winner'].values)
# train_y_ohe = np.zeros((train_y.size, train_y.max()+1))

val_X = val.values[:,:81].reshape((val.shape[0], 9, 9, 1))
val_y = np.array(val['winner'].values)
# val_y_ohe = np.zeros((val_y.size, val_y.max()+1))

In [ ]:
model = Sequential([
    Conv2D(128, kernel_size=(3,3), input_shape=(9,9,1), activation='relu'),
    Dropout(.5),
    Conv2D(64, kernel_size=(3,3), activation='relu'),
    Dropout(.5),
    MaxPooling2D(pool_size=(2, 2), strides=(2, 2)),
#     Activation('relu'),
    Flatten(),
    Dense(15),
    Activation('relu'),
    Dense(3),
    Activation('softmax')
])

In [ ]:
model.compile(optimizer='adam',loss='sparse_categorical_crossentropy')

In [ ]:
model.fit(train_X, train_y, batch_size=64, 
          epochs=5, validation_data=(val_X, val_y))

In [ ]:
preds = model.predict(val_X)
preds_cat = np.argmax(preds, axis=1)

In [ ]:
sums = val.values[:,:81].sum(axis=1)
sums = sums > 30

In [ ]:
(preds_cat== val_y).mean()

In [ ]:
preds_cat